In [23]:
import time
import os
import math

class Graph:
    def __init__(self):
        self.adj_dict = {}
        self.node_dict = {}
        self.edge_list = []
        
    def add_edge(self, v, u) :
        if v not in self.adj_dict:
            self.adj_dict[v] = {u:1}
        else:
            self.adj_dict[v][u] = 1
            
        if u not in self.adj_dict:
            self.adj_dict[u] = {v:1}
        else:
            self.adj_dict[u][v] = 1
            
    def del_edge(self, v, u):
        if u in self.adj_dict[v]:
            del(self.adj_dict[v][u])
            
        if v in self.adj_dict[u]:
            del(self.adj_dict[u][v])
        
    def adj(self, v):
        return self.adj_dict[v]
    
    def vertex(self, v):
        self.adj_list = self.adj_dict[v]
        self.deg = sum(list(map(int, self.adj_list.values())))
        self.vertex_num = 1
        
        return self.vertex_num, self.adj_list, self.deg

def Super_node_generator(v):
    num_v, adj_v, deg_v = gs.vertex(v)
    label_v = {}
    
    # for l in label_kind:
    #     label_v[l] = 0
        
    if v in label_dict:
        label_v[label_dict[v]] = 1
        # for ld_v in label_v:
        #     if ld_v == label_dict[v]:
        #         label_v[ld_v] = 1

    superNode[v] = [num_v, deg_v, adj_v, label_v]
    return superNode[v]

def Super_node_similarity(v, u, n):
    snV = superNode[v]
    snU = superNode[u]
    ew_v = snV[2][u]/snV[1]

    if n != 'None' and (v not in snU[2] or n in snU[2]):
        save_val = snU[2][n]
        del superNode[u][2][n]
        snU[2][v] = save_val
    
    ew_u = snU[2][v]/snU[1]
    n_max = max(snV[0], snU[0])
    d_max = max(snV[1], snU[1])

    lm_v = sum(list(map(int, snV[3].values())))
    lm_u = sum(list(map(int, snU[3].values())))
    dis_att = 0
    
    dis_st = ((abs(snV[0]-snU[0])/n_max) + (abs(snV[1]-snU[1])/d_max) + abs(ew_v - ew_u))/3

    numl = 0
    for vl in snV[3]:
        if vl in snU[3]:
            numl += 1
            dis_att += abs(snV[3][vl] - snU[3][vl])
    
    if numl == 0:
        dis = dis_st/2
    else:
        dis = (dis_st + (dis_att/numl))/2
    sim = 1 - dis
    return sim

def Merged_super_node(v, u):
    num_v = superNode[v][0] + superNode[u][0]
    adj_v = {}
    label_v = {}
    for adj in superNode[v][2]:
        if adj == u:
            continue
        if adj in superNode[u][2]:
            adj_v[adj] = superNode[v][2][adj] + superNode[u][2][adj]
        else:
            adj_v[adj] = superNode[v][2][adj]
            
    for adj in superNode[u][2]:
        if adj == v:
            continue
        if adj not in superNode[v][2]:
            adj_v[adj] = superNode[u][2][adj]
            
    if v in adj_v:
        del adj_v[v]
        
    if u in adj_v:
        del adj_v[u]
        
    deg_v = sum(list(map(int, adj_v.values())))
    
    for lab in superNode[v][3]:
        if lab in superNode[u][3]:
            label_v[lab] = superNode[v][3][lab] + superNode[u][3][lab]
            
    superNode[v] = [num_v, deg_v, adj_v, label_v]
    del(superNode[u])
    
def convert_size(size_bytes):
    if size_bytes == 0:
        return "0B"
    size_name = ("B", "KB", "MB", "GB", "TB", "PB", "EB", "ZB", "YB")
    i = int(math.floor(math.log(size_bytes, 1024)))
    p = math.pow(1024, i)
    s = round(size_bytes / p, 2)
    return "%s %s" % (s, size_name[i])

fileName = 'soc-political-retweet'
# IGPath = './dataset/Amazon/Amazon0302.txt'
# lbPath = './dataset/Amazon/amazon.node_labels'

IGPath = './dataset/' + fileName + '/' + fileName + '.edges' #sys.argv[1]
lbPath = './dataset/' + fileName + '/' + fileName + '.node_labels' #sys.argv[2]
# Separator = '	'
Separator = ','
k = 9225
label_dict = {}
graph_sort = []
label_kind = []
nodeList = []
superNode = {}
lineIndex = 0

s1 = time.time()
label_f = open(lbPath, 'r')
while True:
    line = label_f.readline().replace('\n', '').strip()
    if not line:
        # label_kind.append('None')
        break
    
    lineArray = line.split(Separator)
    if len(lineArray) == 1:
        lineIndex += 1
        node = lineIndex
        label = lineArray[0]
    else:
        node = lineArray[0]
        label = lineArray[1]
    
    label_dict[str(node)] = label
    label_kind.append(label)
label_f.close

label_kind = list(set(label_kind))
edge_f = open(IGPath, 'r')

gs = Graph()
while True:
    line = edge_f.readline().replace('\n', '').strip()
    if not line:
        break

    v = line.split(Separator)[0]
    u = line.split(Separator)[1]
    
    if v == u:
        continue
    
    nodeList.append(v)
    nodeList.append(u)
    
    if v not in label_dict:
        label_dict[v] = 'n'
        
    if u not in label_dict:
        label_dict[u] = 'n'
        
    graph_sort.append([v,u])
    gs.add_edge(v,u)
edge_f.close

nodeList = list(set(nodeList))

for node in nodeList:
    superNode[node] = Super_node_generator(node)
    
sim_list = []

for v, u in graph_sort:
    snSim = Super_node_similarity(v, u, 'None')
    sim_list.append([v, u, snSim]) # triple entry

sim_list.sort(reverse=True, key=lambda x:x[2]) # Sort by similarity

while(True):
    v = sim_list[0][0] # v
    u = sim_list[0][1] # u
    sim = sim_list[0][2] # Edge Similarity
    
    if sim <= 0: # Low Similarity Threshold
        break
        
    if v in superNode and u in superNode:
        del_list = []
        Merged_super_node(v, u) # Marged Super Node v = v + u
        del(sim_list[0])
        
        nei_e = list(superNode[v][2].keys()) # neighber of Node v list

        for sim_one in sim_list:
            if sim_one[0] == v or sim_one[0] == u or sim_one[1] == v or sim_one[1] == u: # List of linked edge deletions in v, u
                del_list.append(sim_one)
                
        for del_one in del_list:
            sim_list.remove(del_one) # Delete connected edges of v, u
            
        for n in nei_e:
            if n not in superNode: # If the node is already deleted
                del superNode[v][2][n]
                continue
            else:
                snSim = Super_node_similarity(v, n, u)
                sim_list.append([v, n, snSim])
    else:
        del(sim_list[0])

    sim_list.sort(reverse=True, key=lambda x:x[2])
    deg_size = len(nei_e)
    sim_size = len(sim_list)
    
    if sim_size <= k: #edge Num
        break
    
Gs = len(sim_list)
Gm = len(superNode)

savefile = './Result/GS4/' + fileName + '_result_GS4.edges'
extension = ['.edges', '.numofnodes']

f = open(savefile + extension[0], 'w')
for e in sim_list:
    str_e = e[0] + ',' + e[1] + ',' + str(e[2]) + '\n'
    f.write(str_e) 
f.close()

f = open(savefile + extension[1], 'w')
for ndata in superNode:
    n = superNode[ndata][0]
    
    str_e = ndata + ',' + str(n) + '\n'
    f.write(str_e)
f.close()
file_size = 0
for e in extension:
    file_size = file_size + os.path.getsize(savefile + e) 
    print(e.split('.')[1], convert_size(os.path.getsize(savefile + e)))
print('OutPut File Size:', convert_size(file_size), 'bytes')

d1 = time.time()-s1
print(sim_size, d1)
print(f'k : {k}, N: {Gm}, E: {Gs}, Time: {d1}')


edges 264.8 KB
numofnodes 57.61 KB
OutPut File Size: 322.42 KB bytes
9225 1114.006650686264
k : 9225, N: 9226, E: 9225, Time: 1114.006650686264


In [6]:
print(sim_list)

[['101', '77', 0.5916529605263159], ['101', '53', 0.5916529605263159], ['101', '87', 0.5916529605263159], ['101', '22', 0.5916529605263159], ['101', '139', 0.5916529605263159], ['101', '116', 0.5916529605263159], ['101', '93', 0.5916529605263159], ['101', '65', 0.5916529605263159], ['101', '31', 0.5916529605263159], ['101', '9', 0.5916529605263159], ['101', '45', 0.5916529605263159], ['101', '39', 0.5916529605263159], ['101', '128', 0.5916529605263159], ['101', '127', 0.5916529605263159], ['101', '168', 0.5916529605263159], ['101', '112', 0.5916529605263159], ['122', '202', 0.5904761904761904], ['101', '13', 0.5669316520467838], ['101', '122', 0.556483004385965], ['101', '119', 0.5174890350877193], ['101', '25', 0.5117735745614036], ['101', '42', 0.5104714912280701], ['101', '160', 0.5061129385964913], ['101', '102', 0.5048108552631578], ['101', '174', 0.5048108552631578], ['101', '173', 0.5048108552631578], ['101', '154', 0.5048108552631578], ['101', '152', 0.5048108552631578], ['101'

In [11]:
print(superNode)

{'199': [1, 1, {'101': 1}, {}], '133': [1, 1, {'101': 1}, {}], '8': [1, 1, {'101': 1}, {}], '17': [1, 1, {'101': 1}, {}], '173': [1, 1, {'101': 1}, {}], '190': [1, 1, {'101': 1}, {}], '70': [1, 1, {'101': 1}, {}], '160': [2, 1, {'101': 1}, {}], '172': [1, 1, {'101': 1}, {}], '139': [1, 2, {'101': 1}, {}], '127': [1, 2, {'101': 1}, {}], '40': [1, 1, {'101': 1}, {}], '66': [1, 1, {'101': 1}, {}], '112': [1, 2, {'101': 1}, {}], '42': [4, 2, {'101': 2}, {}], '78': [1, 1, {'101': 1}, {}], '116': [1, 2, {'101': 1}, {}], '187': [1, 1, {'101': 1}, {}], '181': [1, 1, {'101': 1}, {}], '195': [1, 1, {'101': 1}, {}], '51': [1, 1, {'101': 1}, {}], '141': [1, 1, {'101': 1}, {}], '136': [1, 1, {'101': 1}, {}], '193': [1, 1, {'101': 1}, {}], '218': [1, 1, {'101': 1}, {}], '198': [1, 1, {'101': 1}, {}], '183': [1, 1, {'101': 1}, {}], '128': [1, 2, {'101': 1}, {}], '147': [1, 1, {'101': 1}, {}], '124': [1, 1, {'101': 1}, {}], '144': [1, 1, {'101': 1}, {}], '216': [1, 1, {'101': 1}, {}], '162': [1, 1, {'